In [2]:
import pandas as pd
import vitaldb
import random
import matplotlib.pyplot as plt


pd.set_option('display.max_columns', None)

In [3]:
##Load metadata files
df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # clinical information
df_trks = pd.read_csv("https://api.vitaldb.net/trks")  # track list
df_labs = pd.read_csv('https://api.vitaldb.net/labs')  # laboratory results

In [4]:
df_cases

,caseid,subjectid,casestart,caseend,anestart,aneend,opstart,opend,adm,dis,icu_days,death_inhosp,age,sex,height,weight,bmi,asa,emop,department,optype,dx,opname,approach,position,ane_type,preop_htn,preop_dm,preop_ecg,preop_pft,preop_hb,preop_plt,preop_pt,preop_aptt,preop_na,preop_k,preop_gluc,preop_alb,preop_ast,preop_alt,preop_bun,preop_cr,preop_ph,preop_hco3,preop_be,preop_pao2,preop_paco2,preop_sao2,cormack,airway,tubesize,dltubesize,lmasize,iv1,iv2,aline1,aline2,cline1,cline2,intraop_ebl,intraop_uo,intraop_rbc,intraop_ffp,intraop_crystalloid,intraop_colloid,intraop_ppf,intraop_mdz,intraop_ftn,intraop_rocu,intraop_vecu,intraop_eph,intraop_phe,intraop_epi,intraop_ca
0,1,5955,0,11542,-552,10848.0,1668,10368,-236220,627780,0,0,77.0,M,160.2,67.50,26.3,2.0,0,General surgery,Colorectal,Rectal cancer,Low anterior resection,Open,Lithotomy,General,1,0,Normal Sinus Rhythm,Normal,14.1,189.0,94.0,33.2,141.0,3.1,134.0,4.3,18.0,16.0,10.0,0.82,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,7.5,NaN,NaN,Right forearm,NaN,Left radial,NaN,NaN,NaN,NaN,300.0,0,0,350.0,0,120,0.0,100,70,0,10,0,0,0
1,2,2487,0,15741,-1039,14921.0,1721,14621,-221160,1506840,0,0,54.0,M,167.3,54.80,19.6,2.0,0,General surgery,Stomach,Advanced gastric cancer,Subtotal gastrectomy,Open,Supine,General,0,0,Normal Sinus Rhythm,Normal,10.2,251.0,110.0,31.9,143.0,4.7,88.0,3.8,18.0,15.0,14.0,0.86,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,7.5,NaN,NaN,Left forearm,NaN,NaN,NaN,NaN,NaN,50.0,700.0,0,0,800.0,0,150,0.0,0,100,0,20,0,0,0
2,3,2861,0,4394,-590,4210.0,1090,3010,-218640,40560,0,0,62.0,M,169.1,69.70,24.4,1.0,0,General surgery,Biliary/Pancreas,Gallbladder stone,Cholecystectomy,Videoscopic,Reverse Trendelenburg,General,0,0,Normal Sinus Rhythm,Normal,14.2,373.0,103.0,30.3,144.0,4.9,87.0,4.2,17.0,34.0,14.0,1.18,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,7.5,NaN,NaN,Left forearm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,200.0,0,0,0.0,0,50,0,0,0,0,0
3,4,1903,0,20990,-778,20222.0,2522,17822,-201120,576480,1,0,74.0,M,160.6,53.00,20.5,2.0,0,General surgery,Stomach,Advanced gastric cancer,Distal gastrectomy,Videoscopic,Reverse Trendelenburg,General,1,0,Normal Sinus Rhythm,Normal,14.4,275.0,103.0,34.5,141.0,4.2,108.0,4.1,23.0,18.0,10.0,0.96,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,NaN,NaN,NaN,Left forearm,NaN,Right radial,NaN,NaN,NaN,NaN,270.0,0,0,2700.0,0,80,0.0,100,100,0,50,0,0,0
4,5,4416,0,21531,-1009,22391.0,2591,20291,-67560,3734040,13,0,66.0,M,171.0,59.70,20.4,3.0,1,General surgery,Vascular,Aortic aneurysm,Aneurysmal repair,Open,Prone,General,1,0,Left anterior fascicular block,Normal,10.1,67.0,73.0,36.5,146.0,4.4,126.0,2.6,765.0,77.0,50.0,4.43,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,7.5,NaN,NaN,Right forearm,NaN,Right radial,NaN,Right IJV,NaN,2600.0,1490.0,8,8,7100.0,0,0,0.0,0,160,0,10,900,0,2100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6383,6384,5583,0,15248,-260,15640.0,2140,14140,-215340,648660,0,0,64.0,M,161.5,63.00,24.2,1.0,0,General surgery,Stomach,Advanced gastric cancer,Total gastrectomy,Open,Supine,General,0,0,Normal Sinus Rhythm,Normal,14.5,279.0,109.0,31.0,NaN,NaN,100.0,4.2,31.0,33.0,13.0,0.99,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,7.5,NaN,NaN,Left forearm,NaN,NaN,NaN,NaN,NaN,100.0,330.0,0,0,550.0,0,150,0.0,0,90,0,20,0,0,0
6384,6385,2278,0,20643,-544,20996.0,2396,19496,-225600,1675200,0,0,69.0,M,159.3,62.30,24.6,2.0,0,General surgery,Colorectal,"Colon submucosal tumor, unknown behavior",Ultralow anterior resection,Videoscopic,Lithotomy,General,1,0,Normal Sinus Rhythm,Normal,15.2,239.0,114.0,28.7,144.0,4.0,140.0,3.7,18.0,28.0,19.0,0.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.5,NaN,NaN,Right forearm,Left forearm,Right radial,NaN,NaN,NaN,100.0,250.0,0,0,2500.0,0,100,0.0,0,100,0,25,30,0,300
6385,6386,4045,0,19451,-667,19133.0,3533,18233,-200460,836340,0,0,61.0,F,151.7,43.25,18.8,1.0,0,General surgery,

In [22]:
##Inclusion criteria
caseids = list(
    set(df_trks.loc[df_trks['tname'].str.lower().str.contains("solar8000"), 'caseid']) & 
    set(df_cases.loc[df_cases['age'] > 18, 'caseid']) & 
    set(df_cases.loc[ ~df_cases['ane_type'].str.lower().str.contains("sedationalgesia"), 'caseid'])
)
print(f'{len(caseids)} cases found')

6243 cases found


In [23]:
def setBPMethod(nibpmean, artmean):
    if pd.isna(nibpmean) is True:
        if pd.isna(artmean) is False and artmean >= 0.0:
            return 'ART'
    return 'NIBP'

def getNIBPHypo55(nibpmean, bptype):
    if pd.isna(nibpmean) is False and bptype == 'NIBP':
        if nibpmean < 55.0:
            return 1
    return 0

def getNIBPHypo60(nibpmean, bptype):
    if pd.isna(nibpmean) is False and bptype == 'NIBP':
        if nibpmean < 60.0:
            return 1
    return 0

def getNIBPHypo65(nibpmean, bptype):
    if pd.isna(nibpmean) is False and bptype == 'NIBP':
        if nibpmean < 65.0:
            return 1
    return 0

def getHypotension(bptype, nibp, art):
    if bptype == 'NIBP':
        return nibp
    elif art == 8: #8 ist die Summe von 2 Minuten in 15 sec intervallen und die gleiche anzahl wie bei der rolling funktion
        return 1
    else:
        return 0
    
def setPhase(currentTime, casestart, anestart, opstart, aneend, opend, caseend):
    x = ''
    if currentTime < anestart:
        x = 'pre-airway'
    elif currentTime > anestart and currentTime < opstart:
        x = 'pre-surgery'
    elif currentTime > opstart and currentTime < opend:
        x = 'intra-surgery'
    elif currentTime >= opend:
        x = 'post-surgery'
    else:
        x = 'undefined'
    return x

def getCombinedBP(bptype, nibp, art):
    if bptype == 'NIBP':
        return nibp
    else:
        return art
    
def getRFTPerf(rft20, rft50):
    if pd.isna(rft20) is False and pd.isna(rft50) is True:
        return 'rft20'
    else:
        return 'rft50'
    
def calculatePerfusor(mlperhour, mgperml):
    return mlperhour * mgperml
    

In [24]:
result_df = pd.DataFrame()
track_names = ['Solar8000/HR','Solar8000/PLETH_HR','Solar8000/PLETH_SPO2', 'Solar8000/ETCO2', 'Primus/ETCO2', 'Solar8000/NIBP_SBP', 'Solar8000/NIBP_DBP','Solar8000/NIBP_MBP', 'Solar8000/ART_SBP', 'Solar8000/ART_MBP', 'Solar8000/ART_DBP',
               'Primus/INSP_SEVO', 'Primus/EXP_SEVO','Primus/INSP_DES', 'Primus/EXP_DES', 'Primus/COMPLIANCE', 'Primus/FIO2', 'Primus/PEEP_MBAR', 'Primus/PPLAT_MBAR' , 
               'Primus/MAWP_MBAR', 'Primus/PIP_MBAR', 'Primus/RR_CO2', 'Primus/TV', 'Orchestra/DOBU_RATE', 'Orchestra/EPI_RATE', 'Orchestra/NEPI_RATE', 'Orchestra/PHEN_RATE',
              'Orchestra/PPF20_RATE', 'Orchestra/RFTN20_RATE', 'Orchestra/RFTN50_RATE', 'Orchestra/VASO_RATE']
i = 0
for case in caseids:
    #if case <= 4:
        i+=1
        path = '/Users/stefanbartos/Documents/Studien/Hypotension/VitalDB/vital_files/'+str(case).zfill(4)+'.vital'
        print("Progress: " + str(i/len(caseids)*100))
        vf = vitaldb.VitalFile(path, track_names)
        #working in the vitalFile data format for adding labelling tracks
        
        
        
        #start working with pandas 
        case_df = vf.to_pandas(track_names, 15, return_datetime=True, return_timestamp=True)
        case_df.insert(0, 'caseid', case)
        case_df.insert(2, 'age', df_cases[df_cases['caseid']==case].age.item())
        sex = df_cases[df_cases['caseid']==case].sex.item()
        if sex == 'M':
             case_df.insert(3, 'sex', 1)
        else:
            case_df.insert(3, 'sex', 0)
        case_df.insert(4, 'asa', df_cases[df_cases['caseid']==case].asa.item())
        if df_cases[df_cases['caseid']==case].emop.item() == 0:
            case_df.insert(5, 'urgency', 1)
        else:
            case_df.insert(5, 'urgency', 3)
        department = df_cases[df_cases['caseid']==case].department.item()
        if department == 'General surgery':
            case_df.insert(6, 'surgery_type', 0)
        elif department == 'Thoracic surgery':
            case_df.insert(6, 'surgery_type', 16)
        elif department == 'Urology':
            case_df.insert(6, 'surgery_type', 10)
        elif department == 'Gynecology':
            case_df.insert(6, 'surgery_type', 4)
        else:
            case_df.insert(6, 'surgery_type', 17)
        case_df.insert(7, 'casestart', df_cases[df_cases['caseid']==case].casestart.item())
        case_df.insert(8, 'caseend', df_cases[df_cases['caseid']==case].caseend.item())
        case_df.insert(9, 'anestart', df_cases[df_cases['caseid']==case].anestart.item())
        case_df.insert(10, 'aneend', df_cases[df_cases['caseid']==case].aneend.item())
        case_df.insert(11, 'opstart', df_cases[df_cases['caseid']==case].opstart.item())
        case_df.insert(12, 'opend', df_cases[df_cases['caseid']==case].opend.item())
        case_df['BPType'] = case_df.apply(lambda x: setBPMethod(x['Solar8000/NIBP_MBP'],x['Solar8000/ART_MBP']), axis=1)
        #add new combined blood pressure column for Lorenz depending on flag in BPType
        case_df['systolicbp'] = case_df.apply(lambda x: getCombinedBP(x['BPType'], x['Solar8000/NIBP_SBP'], x['Solar8000/ART_SBP'] ), axis=1)
        case_df['diastolicbp'] = case_df.apply(lambda x: getCombinedBP(x['BPType'], x['Solar8000/NIBP_DBP'], x['Solar8000/ART_DBP'] ), axis=1)
        case_df['meanbp'] = case_df.apply(lambda x: getCombinedBP(x['BPType'], x['Solar8000/NIBP_MBP'], x['Solar8000/ART_MBP'] ), axis=1)
        #add new indicator which remifentanil column to use for the final combined column
        case_df['rft_dosing'] = case_df.apply(lambda x: getRFTPerf(x['Orchestra/RFTN20_RATE'], x['Orchestra/RFTN50_RATE']), axis=1)
        #calculate the motor pumps in mg/h at the current time 
        case_df['propofol_perfusor'] = case_df.apply(lambda x: calculatePerfusor(x['Orchestra/PPF20_RATE'], 20), axis=1)
        case_df['remifentanil_perfusor'] = case_df.apply(lambda x: calculatePerfusor(x['Orchestra/RFTN20_RATE'], 0.02) if x['rft_dosing'] == 'rft20' else calculatePerfusor(x['Orchestra/RFTN50_RATE'], 0.05) , axis=1)
        case_df['epinephrin_perfusor'] = case_df.apply(lambda x: calculatePerfusor(x['Orchestra/EPI_RATE'], 0.02), axis=1)
        case_df['noradrenalin_perfusor'] = case_df.apply(lambda x: calculatePerfusor(x['Orchestra/NEPI_RATE'], 0.02), axis=1)
        case_df['phenylephrin_perfusor'] = case_df.apply(lambda x: calculatePerfusor(x['Orchestra/PHEN_RATE'], 0.1), axis=1)
        case_df['vasopressin_perfusor'] = case_df.apply(lambda x: calculatePerfusor(x['Orchestra/VASO_RATE'], 0.2), axis=1) #cave Dosing 0.2 Units/ml
        #create hypotensionLabel 55mean
        case_df['NIBP55'] = case_df.apply(lambda x: getNIBPHypo55(x['Solar8000/NIBP_MBP'], x['BPType']), axis=1)
        case_df['artMBP'] = case_df['Solar8000/ART_MBP']
        case_df['ART55Helper'] = case_df.apply(lambda x: 1 if x['artMBP'] < 55 else 0, axis=1)
        case_df['ART55'] = case_df.rolling(window = 8, min_periods=8).ART55Helper.sum()
        case_df['hyptotension55'] = case_df.apply(lambda x: getHypotension(x['BPType'], x['NIBP55'],x['ART55']), axis=1)
        #create HypotensionLabel 60mean
        case_df['NIBP60'] = case_df.apply(lambda x: getNIBPHypo60(x['Solar8000/NIBP_MBP'], x['BPType']), axis=1)
        case_df['artMBP'] = case_df['Solar8000/ART_MBP']
        case_df['ART60Helper'] = case_df.apply(lambda x: 1 if x['artMBP'] < 60 else 0, axis=1)
        case_df['ART60'] = case_df.rolling(window = 8, min_periods=8).ART60Helper.sum()
        case_df['hyptotension60'] = case_df.apply(lambda x: getHypotension(x['BPType'], x['NIBP60'],x['ART60']), axis=1)
        #create HypotensionLabel 65mean
        case_df['NIBP65'] = case_df.apply(lambda x: getNIBPHypo60(x['Solar8000/NIBP_MBP'], x['BPType']), axis=1)
        case_df['artMBP'] = case_df['Solar8000/ART_MBP']
        case_df['ART65Helper'] = case_df.apply(lambda x: 1 if x['artMBP'] < 65 else 0, axis=1)
        case_df['ART65'] = case_df.rolling(window = 8, min_periods=8).ART65Helper.sum()
        case_df['hyptotension65'] = case_df.apply(lambda x: getHypotension(x['BPType'], x['NIBP65'],x['ART65']), axis=1)
        #create spO2 Label
        case_df['spo2low'] = case_df.apply(lambda x: 1 if x['Solar8000/PLETH_SPO2'] < 88 else 0, axis=1)
        case_df['spo2sum'] = case_df.rolling(window = 8, min_periods=8).spo2low.sum()
        case_df['hypoxemia'] = case_df.apply(lambda x: 1 if x['spo2sum'] == 8 else 0, axis=1)
        #convert seconds to datetime
        case_df['casestartTime'] = pd.to_datetime(case_df['casestart'], unit='s',origin=pd.Timestamp('2100-01-01'))
        case_df['anestartTime'] = pd.to_datetime(abs(case_df['anestart']), unit='s',origin=pd.Timestamp('2100-01-01'))
        case_df['opstartTime'] = pd.to_datetime(case_df['opstart'], unit='s',origin=pd.Timestamp('2100-01-01'))
        case_df['aneendTime'] = pd.to_datetime(case_df['aneend'], unit='s',origin=pd.Timestamp('2100-01-01'))
        case_df['opendTime'] = pd.to_datetime(case_df['opend'], unit='s',origin=pd.Timestamp('2100-01-01'))
        case_df['caseendTime'] = pd.to_datetime(case_df['caseend'], unit='s',origin=pd.Timestamp('2100-01-01'))
        #set Phase label
        case_df['Time'] = case_df['Time'].dt.tz_localize(None)
        case_df['phase'] = case_df.apply(lambda x: setPhase(x['Time'],x['casestartTime'],x['anestartTime'],x['opstartTime'],x['aneendTime'],x['opendTime'],x['caseendTime']), axis=1)
        result_df = pd.concat([result_df, case_df], ignore_index=True)
    #else:
     #   break	

Progress: 0.016017940092904054
Progress: 0.03203588018580811
Progress: 0.04805382027871216
Progress: 0.06407176037161622
Progress: 0.08008970046452026
Progress: 0.09610764055742432
Progress: 0.11212558065032836
Progress: 0.12814352074323243
Progress: 0.14416146083613646
Progress: 0.1601794009290405
Progress: 0.1761973410219446
Progress: 0.19221528111484865
Progress: 0.20823322120775267
Progress: 0.22425116130065673
Progress: 0.2402691013935608
Progress: 0.25628704148646486
Progress: 0.2723049815793689
Progress: 0.2883229216722729
Progress: 0.304340861765177
Progress: 0.320358801858081
Progress: 0.3363767419509851
Progress: 0.3523946820438892
Progress: 0.3684126221367932
Progress: 0.3844305622296973
Progress: 0.4004485023226013
Progress: 0.41646644241550534
Progress: 0.4324843825084094
Progress: 0.44850232260131345
Progress: 0.46452026269421753
Progress: 0.4805382027871216
Progress: 0.4965561428800256
Progress: 0.5125740829729297
Progress: 0.5285920230658337
Progress: 0.5446099631587378

In [25]:
#result_df.head(50)

In [26]:
some_dict = {
    'caseid' : 'rand_id',
    'Time' : 'case_time',
    'age' : 'age',
    'sex' : 'gender',
    'asa' : 'asa_score',
    'urgency' : 'urgency',
    'surgery_type' : 'surgery_type',
    'Solar8000/HR' : 'hf',
    'Solar8000/PLETH_HR' : 'pulse',
    'Solar8000/PLETH_SPO2' : 'spo2',
    'Primus/ETCO2' : 'etco2',
    'BPType' : 'invasivebp',
    'Primus/INSP_SEVO' : 'insevo',
    'Primus/EXP_SEVO' : 'exsevo',
    'Primus/INSP_DES' : 'indes',
    'Primus/EXP_DES' : 'exdes',
    'Primus/COMPLIANCE' : 'compliance',
    'Primus/FIO2' : 'fio2',
    'Primus/PEEP_MBAR' : 'peep',
    'Primus/PPLAT_MBAR' : 'plateau',
    'Primus/PIP_MBAR' : 'ppeak',
    'Primus/MAWP_MBAR' : 'pmean',
    'phase' : 'phase',
    'hyptotension60' : 'hypotension60',
    'hyptotension55' : 'hypotension55',
    'hyptotension65' : 'hypotension65',
    'hypoxemia' : 'hypoxemia'
}

result_df.rename(columns=some_dict, inplace=True)

In [27]:
#result_df['Time'] = result_df['Time'].dt.tz_localize(None)
#result_df.to_excel('export.xlsx')
result_df.to_parquet('exportVitalDB_all_V2.parquet')

In [18]:
#result_df['Orchestra/RFTN20_RATE'] = result_df['Orchestra/RFTN20_RATE'].fillna(method="ffill")

In [7]:

result_df = pd.read_parquet('exportVitalDB_all')

In [28]:
result_df

,rand_id,case_time,age,gender,asa_score,urgency,surgery_type,casestart,caseend,anestart,aneend,opstart,opend,hf,pulse,spo2,Solar8000/ETCO2,etco2,Solar8000/NIBP_SBP,Solar8000/NIBP_DBP,Solar8000/NIBP_MBP,Solar8000/ART_SBP,Solar8000/ART_MBP,Solar8000/ART_DBP,insevo,exsevo,indes,exdes,compliance,fio2,peep,plateau,pmean,ppeak,Primus/RR_CO2,Primus/TV,Orchestra/DOBU_RATE,Orchestra/EPI_RATE,Orchestra/NEPI_RATE,Orchestra/PHEN_RATE,Orchestra/PPF20_RATE,Orchestra/RFTN20_RATE,Orchestra/RFTN50_RATE,Orchestra/VASO_RATE,invasivebp,systolicbp,diastolicbp,meanbp,rft_dosing,propofol_perfusor,remifentanil_perfusor,epinephrin_perfusor,noradrenalin_perfusor,phenylephrin_perfusor,vasopressin_perfusor,NIBP55,artMBP,ART55Helper,ART55,hypotension55,NIBP60,ART60Helper,ART60,hypotension60,NIBP65,ART65Helper,ART65,hypotension65,spo2low,spo2sum,hypoxemia,casestartTime,anestartTime,opstartTime,aneendTime,opendTime,caseendTime,phase
0,1,2100-01-01 00:00:02.765000,77.0,1,2.0,1,0,0,11542,-552,10848.0,1668,10368,90.0,90.0,97.0,NaN,0.0,NaN,NaN,NaN,8.0,-9.0,-12.0,0.0,0.0,NaN,NaN,NaN,100.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,NaN,NaN,NaN,rft50,NaN,NaN,NaN,NaN,NaN,NaN,0,-9.0,1,NaN,0,0,1,NaN,0,0,1,NaN,0,0,NaN,0,2100-01-01,2100-01-01 00:09:12,2100-01-01 00:27:48,2100-01-01 03:00:48,2100-01-01 02:52:48,2100-01-01 03:12:22,pre-airway
1,1,2100-01-01 00:00:17.765000,77.0,1,2.0,1,0,0,11542,-552,10848.0,1668,10368,78.0,88.0,97.0,NaN,0.0,NaN,NaN,NaN,-9.0,-9.0,-9.0,0.0,0.0,NaN,NaN,NaN,100.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,NaN,NaN,NaN,rft50,NaN,NaN,NaN,NaN,NaN,NaN,0,-9.0,1,NaN,0,0,1,NaN,0,0,1,NaN,0,0,NaN,0,2100-01-01,2100-01-01 00:09:12,2100-01-01 00:27:48,2100-01-01 03:00:48,2100-01-01 02:52:48,2100-01-01 03:12:22,pre-airway
2,1,2100-01-01 00:00:32.765000,77.0,1,2.0,1,0,0,11542,-552,10848.0,1668,10368,78.0,86.0,96.0,NaN,0.0,NaN,NaN,NaN,-8.0,-8.0,-8.0,0.0,0.0,NaN,NaN,NaN,100.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,NaN,NaN,NaN,rft50,NaN,NaN,NaN,NaN,NaN,NaN,0,-8.0,1,NaN,0,0,1,NaN,0,0,1,NaN,0,0,NaN,0,2100-01-01,2100-01-01 00:09:12,2100-01-01 00:27:48,2100-01-01 03:00:48,2100-01-01 02:52:48,2100-01-01 03:12:22,pre-airway
3,1,2100-01-01 00:00:47.765000,77.0,1,2.0,1,0,0,11542,-552,10848.0,1668,10368,89.0,91.0,98.0,NaN,0.0,NaN,NaN,NaN,-9.0,-9.0,-9.0,0.0,0.0,NaN,NaN,NaN,100.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,NaN,NaN,NaN,rft50,NaN,NaN,NaN,NaN,NaN,NaN,0,-9.0,1,NaN,0,0,1,NaN,0,0,1,NaN,0,0,NaN,0,2100-01-01,2100-01-01 00:09:12,2100-01-01 00:27:48,2100-01-01 03:00:48,2100-01-01 02:52:48,2100-01-01 03:12:22,pre-airway
4,1,2100-01-01 00:01:02.765000,77.0,1,2.0,1,0,0,11542,-552,10848.0,1668,10368,85.0,86.0,97.0,NaN,0.0,NaN,NaN,NaN,-9.0,-9.0,-9.0,0.0,0.0,NaN,NaN,NaN,100.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,NaN,NaN,NaN,rft50,NaN,NaN,NaN,NaN,NaN,NaN,0,-9.0,1,NaN,0,0,1,NaN,0,0,1,NaN,0,0,NaN,0,2100-01-01,2100-01-01 00:09:12,2100-01-01 00:27:48,2100-01-01 03:00:48,2100-01-01 02:52:48,2100-01-01 03:12:22,pre-airway
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4724091,6388,2100-01-01 02:40:17.693300,47.0,0,2.0,1,4,0,10249,-79,10121.0,2321,9221,79.0,79.0,100.0,8.0,33.0,112.0,73.0,88.0,119.0,93.0,73.0,0.0,0.0,NaN,NaN,31.4,92.0,NaN,NaN,0.0,4.0,8.0,325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,112.0,73.0,88.0,rft50,NaN,NaN,NaN,NaN,NaN,NaN,0,93.0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2100-01-01,2100-01-01 00:01:19,2100-01-01 00:38:41,2100-01-01 02:48:41,2100-01-01 02:33:41,2100-01-01 02:50:49,post-surgery
4724092,6388,2100-01-01 02:40:32.693300,47.0,0,2.0,1,4,0,10249,-79,10121.0,2321,9221,79.0,80.0,100.0,33.0,23.0,112.0,73.0,88.0,119.0,93.0,71.0,0.0,0.2,NaN,NaN,31.4,94.0,NaN,NaN,0.0,4.0,14.0,256.0,NaN,NaN,NaN,NaN,Na

In [13]:
result_df.columns

Index(['rand_id', 'case_time', 'age', 'gender', 'asa_score', 'urgency',
       'surgery_type', 'casestart', 'caseend', 'anestart', 'aneend', 'opstart',
       'opend', 'hf', 'pulse', 'spo2', 'Solar8000/ETCO2', 'etco2',
       'Solar8000/NIBP_SBP', 'Solar8000/NIBP_DBP', 'Solar8000/NIBP_MBP',
       'Solar8000/ART_SBP', 'Solar8000/ART_MBP', 'Solar8000/ART_DBP', 'insevo',
       'exsevo', 'indes', 'exdes', 'compliance', 'fio2', 'peep', 'plateau',
       'pmean', 'ppeak', 'Primus/RR_CO2', 'Primus/TV', 'Orchestra/DOBU_RATE',
       'epinephrin_perfusor', 'noradrenalin_perfusor', 'phenylephrin_perfusor',
       'propofol_perfusor', 'remifentanil_perfusor', 'Orchestra/RFTN50_RATE',
       'vasopressin_perfusor', 'invasivebp', 'NIBP55', 'artMBP', 'ART55Helper',
       'ART55', 'hypotension55', 'NIBP60', 'ART60Helper', 'ART60',
       'hypotension60', 'NIBP65', 'ART65Helper', 'ART65', 'hypotension65',
       'spo2low', 'spo2sum', 'hypoxemia', 'casestartTime', 'anestartTime',
       'opstartTime

In [14]:
result_df

,rand_id,case_time,age,gender,asa_score,urgency,surgery_type,casestart,caseend,anestart,aneend,opstart,opend,hf,pulse,spo2,Solar8000/ETCO2,etco2,Solar8000/NIBP_SBP,Solar8000/NIBP_DBP,Solar8000/NIBP_MBP,Solar8000/ART_SBP,Solar8000/ART_MBP,Solar8000/ART_DBP,insevo,exsevo,indes,exdes,compliance,fio2,peep,plateau,pmean,ppeak,Primus/RR_CO2,Primus/TV,Orchestra/DOBU_RATE,epinephrin_perfusor,noradrenalin_perfusor,phenylephrin_perfusor,propofol_perfusor,remifentanil_perfusor,Orchestra/RFTN50_RATE,vasopressin_perfusor,invasivebp,NIBP55,artMBP,ART55Helper,ART55,hypotension55,NIBP60,ART60Helper,ART60,hypotension60,NIBP65,ART65Helper,ART65,hypotension65,spo2low,spo2sum,hypoxemia,casestartTime,anestartTime,opstartTime,aneendTime,opendTime,caseendTime,phase
0,1,2100-01-01 00:00:02.765000,77.0,1,2.0,1,0,0,11542,-552,10848.0,1668,10368,90.0,90.0,97.0,NaN,0.0,NaN,NaN,NaN,8.0,-9.0,-12.0,0.0,0.0,NaN,NaN,NaN,100.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,0,-9.0,1,NaN,0,0,1,NaN,0,0,1,NaN,0,0,NaN,0,2100-01-01,2100-01-01 00:09:12,2100-01-01 00:27:48,2100-01-01 03:00:48,2100-01-01 02:52:48,2100-01-01 03:12:22,pre-airway
1,1,2100-01-01 00:00:17.765000,77.0,1,2.0,1,0,0,11542,-552,10848.0,1668,10368,78.0,88.0,97.0,NaN,0.0,NaN,NaN,NaN,-9.0,-9.0,-9.0,0.0,0.0,NaN,NaN,NaN,100.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,0,-9.0,1,NaN,0,0,1,NaN,0,0,1,NaN,0,0,NaN,0,2100-01-01,2100-01-01 00:09:12,2100-01-01 00:27:48,2100-01-01 03:00:48,2100-01-01 02:52:48,2100-01-01 03:12:22,pre-airway
2,1,2100-01-01 00:00:32.765000,77.0,1,2.0,1,0,0,11542,-552,10848.0,1668,10368,78.0,86.0,96.0,NaN,0.0,NaN,NaN,NaN,-8.0,-8.0,-8.0,0.0,0.0,NaN,NaN,NaN,100.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,0,-8.0,1,NaN,0,0,1,NaN,0,0,1,NaN,0,0,NaN,0,2100-01-01,2100-01-01 00:09:12,2100-01-01 00:27:48,2100-01-01 03:00:48,2100-01-01 02:52:48,2100-01-01 03:12:22,pre-airway
3,1,2100-01-01 00:00:47.765000,77.0,1,2.0,1,0,0,11542,-552,10848.0,1668,10368,89.0,91.0,98.0,NaN,0.0,NaN,NaN,NaN,-9.0,-9.0,-9.0,0.0,0.0,NaN,NaN,NaN,100.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,0,-9.0,1,NaN,0,0,1,NaN,0,0,1,NaN,0,0,NaN,0,2100-01-01,2100-01-01 00:09:12,2100-01-01 00:27:48,2100-01-01 03:00:48,2100-01-01 02:52:48,2100-01-01 03:12:22,pre-airway
4,1,2100-01-01 00:01:02.765000,77.0,1,2.0,1,0,0,11542,-552,10848.0,1668,10368,85.0,86.0,97.0,NaN,0.0,NaN,NaN,NaN,-9.0,-9.0,-9.0,0.0,0.0,NaN,NaN,NaN,100.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,0,-9.0,1,NaN,0,0,1,NaN,0,0,1,NaN,0,0,NaN,0,2100-01-01,2100-01-01 00:09:12,2100-01-01 00:27:48,2100-01-01 03:00:48,2100-01-01 02:52:48,2100-01-01 03:12:22,pre-airway
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4724091,6388,2100-01-01 02:40:17.693300,47.0,0,2.0,1,4,0,10249,-79,10121.0,2321,9221,79.0,79.0,100.0,8.0,33.0,112.0,73.0,88.0,119.0,93.0,73.0,0.0,0.0,NaN,NaN,31.4,92.0,NaN,NaN,0.0,4.0,8.0,325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,0,93.0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2100-01-01,2100-01-01 00:01:19,2100-01-01 00:38:41,2100-01-01 02:48:41,2100-01-01 02:33:41,2100-01-01 02:50:49,post-surgery
4724092,6388,2100-01-01 02:40:32.693300,47.0,0,2.0,1,4,0,10249,-79,10121.0,2321,9221,79.0,80.0,100.0,33.0,23.0,112.0,73.0,88.0,119.0,93.0,71.0,0.0,0.2,NaN,NaN,31.4,94.0,NaN,NaN,0.0,4.0,14.0,256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,0,93.0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2100-01-01,2100-01-01 00:01:19,2100-01-01 00:38:41,2100-01-01 02:48:41,2100-01-01 02:33:41,2100-01-01 02:50:49,post-surgery
4724093,6388,2100-01-01 02:40:47.693300,47.0,0,2.0,1,4,0,10249,-79,10121.0,2321,9221,NaN,300.0,100.0,23.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.2,NaN,NaN,31.4,94.0,NaN,NaN,0.0,NaN,11.0,256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIBP,0,NaN,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2100-0